NARENDRA SINGH BISHT

AM.EN.P2ARI20043

M.Tech. AI (Semester I)

**You start with the sequence ABABAECCEC, or in general any sequence made from A,B, C, and E. You can transform this sequence using the following equalities: AC = E,AB = BC, BB = E, CC = E, and E x = x for any x. For example, ABBC can be transformed into AEC, and then AC, and then E.**

**Your goal is to produce the sequence E.**

**Define an admissible heuristic for the problem and write an A* algorithm to solve the problem.**

In [1]:
class Priority_Queue:
    
    def __init__(self):
        self.queue = []
    
    def enqueue(self, node):
        self.queue.append(node)
        
    def dequeue(self):
        if not self.empty():
            max_priority_idx = 0
            for i in range(len(self.queue)): 
                if self.queue[i].priority > self.queue[max_priority_idx].priority : 
                    max_priority_idx = i 
            item = self.queue[max_priority_idx] 
            del self.queue[max_priority_idx] 
            return item 
        else:
            raise Exception("Priority_Queue Empty!")
            
    def empty(self):
        return len(self.queue) == 0
    
    def contains_state(self, state):
        return any(node.state == state for node in self.queue)
    
    def __str__(self):
        result = f"Number of items in queue = {len(self.queue)}\n"
        for item in self.queue:
            result += f"{item}\n"
        return result    

In [2]:
class Node:
    def __init__(self, state, parent, action, depth, priority=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.depth = depth
        self.priority = priority
    
    def repeated_state(self):
        if self.parent == None or self.parent.parent == None: 
            return False
        if self.parent.parent.state.equals(self.state): 
            return True
        return False
    
    def __str__(self):
        result = f"{self.state}"
        result += f" {self.depth}"
        result += f" {self.priority}"
        if self.parent != None:
            result += f" {self.parent.state}"
            result += f" {self.action}"
        return result  

In [3]:
class Search:
    
    # A* Search
    
    def __init__(self, start_state, goal_state):

        self.start_state = start_state
        self.goal_state = goal_state
            
    def heuristic(self, node):      
        
        def get_count(sub):
            c = seq.count(sub)
            if c != -1: return c
            return 0
        
        seq = node[0].sequence
        
        #Trivial Heuristic
        #return 0
        
        #Admissible Heuristic
        return get_count("AC") + get_count("BB") + get_count("CC")
    
    def find_solution(self):
        
        # Initialize fringe
        start_node = Node(self.start_state, None, None, 0)
        fringe = Priority_Queue()
        fringe.enqueue(start_node)
        
        # Initialize an empty explored set
        self.explored = set()
        
        while not fringe.empty():
            
            # Choose a node from the fringe
            current_node = fringe.dequeue()
            
            if not current_node.state.legal():
                return None
            
            # If the chosen node is the goal, then we have a solution
            if self.goal_state.equals(current_node.state):
                print(f"No. of nodes explored = {len(self.explored)}")
                return current_node

            # Mark node as explored
            self.explored.add(current_node.state)
            
            # Add possible successors to the fringe
            successors = current_node.state.find_successors()
            for successor in successors:   
                if not fringe.contains_state(successor[0]) and successor[0] not in self.explored:
                    n = Node(successor[0],#state
                             current_node,
                             successor[1],#action
                             current_node.depth+1, current_node.depth + 1 + self.heuristic(successor))
                    if n.repeated_state():
                        del(n)
                    else:
                        fringe.enqueue(n)
            #print(fringe)
        return None
    
    def build_path(self, node):
        result = []
        while node:
            result.insert(0, node)
            node = node.parent
        return result
    
    def show_path(self, node):
        path = self.build_path(node)        
        for current_node in path:
            if current_node.action:
                print(current_node.action)
            print(current_node.state)
        print(f"Number of steps = {current_node.depth}")
    
    def solve(self):
        solution = self.find_solution()
        if solution == None:
            print("Search failed")
        else:
            print("Search completed successfully!")
            self.show_path(solution)

In [4]:
def replace_nth(s, old, new, n):
    count = 0

    find = s.find(old)
    if find != -1: count += 1
        
    while count != n and find != -1:
        find = s.find(old, find + 1)
        if find != -1: count += 1
            
    if count == n:
        return s[:find] + new + s[find + len(old):]

In [5]:
class Sequence_State:

    def __init__(self, sequence):
        self.sequence = sequence
       
    def __str__(self):    
        return self.sequence
    
    def equals(self, state):
        return (self.sequence == state.sequence)
    
    def legal(self):
        allowed = {'A', 'B', 'C', 'E'}
        current = set(self.sequence)
        if current.issubset(allowed):
            return True
        return False
    
    def apply_helper(self, successors, old, new, op):
        number_of_occurrences = self.sequence.count(old)
        count = 1
        while(count <= number_of_occurrences) :
            s = Sequence_State(replace_nth(self.sequence, old, new, count))
            if s.sequence != None: successors.append((s, op))
            count += 1
        
    
    # There are five possible transformations:
    # 1. AC = E
    def apply_AC_E(self, successors):
        self.apply_helper(successors, "AC", "E", "AC = E")
    
    # 2. AB = BC
    def apply_AB_BC(self, successors):
        self.apply_helper(successors, "AB", "BC", "AB = BC")
    
    # 3. BB = E
    def apply_BB_E(self, successors):
        self.apply_helper(successors, "BB", "E", "BB = E")
    
    # 4. CC = E
    def apply_CC_E(self, successors):
        self.apply_helper(successors, "CC", "E", "CC = E")
    
    # 5. Ex = x for any x
    def apply_Ex_x(self, successors):
        self.apply_helper(successors, "EA", "A", "Ex = x (EA = A)")
        self.apply_helper(successors, "EB", "B", "Ex = x (EB = B)")
        self.apply_helper(successors, "EC", "C", "Ex = x (EC = C)")
        self.apply_helper(successors, "EE", "E", "Ex = x (EE = E)")
     
    def find_successors(self):
        successors = []
        self.apply_AC_E(successors)
        self.apply_AB_BC(successors)
        self.apply_BB_E(successors)
        self.apply_CC_E(successors)
        self.apply_Ex_x(successors)
        return successors

In [6]:
s = Search(Sequence_State("ABBC"), Sequence_State("E"))
s.solve()

No. of nodes explored = 4
Search completed successfully!
ABBC
BB = E
AEC
Ex = x (EC = C)
AC
AC = E
E
Number of steps = 3


In [7]:
s = Search(Sequence_State("ABABAECCEC"), Sequence_State("E"))
s.solve()

No. of nodes explored = 11
Search completed successfully!
ABABAECCEC
AB = BC
ABBCAECCEC
Ex = x (EC = C)
ABBCACCEC
Ex = x (EC = C)
ABBCACCC
AC = E
ABBCECC
Ex = x (EC = C)
ABBCCC
BB = E
AECCC
Ex = x (EC = C)
ACCC
AC = E
ECC
Ex = x (EC = C)
CC
CC = E
E
Number of steps = 10


In [8]:
# Illegal character in the string
s = Search(Sequence_State("ABBCX"), Sequence_State("E"))
s.solve()

Search failed


In [9]:
s = Search(Sequence_State("ABABAECCECABBC"), Sequence_State("E")) 
s.solve()

No. of nodes explored = 18
Search completed successfully!
ABABAECCECABBC
AB = BC
ABBCAECCECABBC
Ex = x (EC = C)
ABBCACCECABBC
Ex = x (EC = C)
ABBCACCCABBC
AC = E
ABBCECCABBC
Ex = x (EC = C)
ABBCCCABBC
BB = E
AECCCABBC
Ex = x (EC = C)
ACCCABBC
AC = E
ECCABBC
Ex = x (EC = C)
CCABBC
BB = E
CCAEC
Ex = x (EC = C)
CCAC
CC = E
EAC
Ex = x (EA = A)
AC
AC = E
E
Number of steps = 14


In [10]:
s = Search(Sequence_State("ABABAECCECABABAECCEC"), Sequence_State("E")) 
s.solve()

No. of nodes explored = 24
Search completed successfully!
ABABAECCECABABAECCEC
AB = BC
ABBCAECCECABABAECCEC
AB = BC
ABBCAECCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCC
AC = E
ABBCECCABBCACCC
Ex = x (EC = C)
ABBCCCABBCACCC
AC = E
ABBCCCABBCECC
Ex = x (EC = C)
ABBCCCABBCCC
BB = E
AECCCABBCCC
Ex = x (EC = C)
ACCCABBCCC
AC = E
ECCABBCCC
Ex = x (EC = C)
CCABBCCC
BB = E
CCAECCC
Ex = x (EC = C)
CCACCC
AC = E
CCECC
Ex = x (EC = C)
CCCC
CC = E
ECC
Ex = x (EC = C)
CC
CC = E
E
Number of steps = 21


In [11]:
s = Search(Sequence_State("ABABAECCECABABAECCECABABAECCECABABAECCEC"), Sequence_State("E")) 
s.solve()

No. of nodes explored = 53
Search completed successfully!
ABABAECCECABABAECCECABABAECCECABABAECCEC
AB = BC
ABBCAECCECABABAECCECABABAECCECABABAECCEC
AB = BC
ABBCAECCECABBCAECCECABABAECCECABABAECCEC
AB = BC
ABBCAECCECABBCAECCECABBCAECCECABABAECCEC
AB = BC
ABBCAECCECABBCAECCECABBCAECCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCAECCECABBCAECCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCACCECABBCAECCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCACCECABBCACCECABBCAECCEC
Ex = x (EC = C)
ABBCACCECABBCACCECABBCACCECABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCECABBCACCECABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCCABBCACCECABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCCABBCACCCABBCACCEC
Ex = x (EC = C)
ABBCACCCABBCACCCABBCACCCABBCACCC
AC = E
ABBCECCABBCACCCABBCACCCABBCACCC
Ex = x (EC = C)
ABBCCCABBCACCCABBCACCCABBCACCC
AC = E
ABBCCCABBCECCABBCACCCABBCACCC
Ex = x (EC = C)
ABBCCCABBCCCABBCACCCABBCACCC
AC = E
ABBCCCABBCCCABBCECCABBCACCC
Ex = x (EC = C)
ABBCCCABBCCCABBCCCABBCACCC
AC = E
ABBCCCABBCCCABBCCCABBCECC
Ex = x